# Converting a Normal EPANET .inp File to a SWMM .inp file assuming Volume-Restricted Withdrawal (Outlet-Storage)
This notebook takes an input EPANET file with demands input normally as a CWS base demand and outputs a .inp file configured for SWMM and uses a volume-restricted assumption  
A simplified schematic of the modified demand node in this method (Outlet-Storage) is seen below:  



### First, we import the necessary libraries and packages
**WNTR** for building EPANET network models in Python  
**NUMPY & PANDAS** for data handling and processing  
**re** for searching and matching text in the .inp file using regular expressions

In [6]:
import wntr
import numpy as np 
import pandas as pd
import re
import math

### Specifying paths for EPANET.inp File to be Converted and preprocessing the input
**Warning:** *Paths in this script (and the rest of this repo) are absolute unless running the network files provided within the repo*  
Input filename (with extensions) as string.  
For running the .inp files in this repository, you can use this relative path `"../Network-Files/Network X/"` where X is the network number 

In [7]:
 # Replace with appropriate path and filename
directory=r"C:\IWS_Modelling\Github\IWS-Modelling-Methods-Repo\Network-Files\Network 2"
filename=r'\Network2_12hr_PDA.inp'
name_only=filename.rsplit('_',maxsplit=1)[0]
print("Selected File: ",name_only)
abs_path=directory+filename

Selected File:  \Network2_12hr


### Necessary Assumptions Input
Converting a CWS demand-driven analysis into an IWS pressure-driven analysis requires some assumptions in all methods  
The resistance of the service connection between the demand junction and the household (end-user) is uncertain and is modelled using two assumptions  
The **desired head (pressure)** is the pressure at the demand junction at which (or above) the consumer can satisfy their full demand in the supply duration (or possible less)  
The **minimum head (pressure)** is the minimum pressure at the demand junction required for flow to begin passing through the service connection  
These two assumptions dictate the flow-pressure relationship that determines the pressure-dependent flow through the service connection as follows:

$$ Q\, = \!Q_{des}\sqrt{\frac{H_{j}-H^{min}}{H^{des}-H^{min}}} \quad[1]$$ 
Where Q is the flow through the service connection, $Q_{des}$ is the desired (base) demand, $H_j$ is the head at the demand junction $j$, $H^{min}$ is the minimum head, and $H^{des}$ is the desired head


In [8]:
desired_pressure=10     # Set the desired pressure
minimum_pressure=0      # Set the minimum pressure
pressure_diff=desired_pressure-minimum_pressure  

### Extracting information from the EPANET file
To modify the .inp file, demand junction IDs, elevations, x and y coordinates  
We use wntr to build the network model of the input file and use wntr's junctions module to extract the details of each node

In [9]:
demand_nodes=[]       # For storing list of nodes that have non-zero demands
desired_demands=[]    # For storing demand rates desired by each node for desired volume calculations
elevations=[]         # For storing elevations of demand nodes
coords=dict()         # For storing coordinates corresponding to each node as a tuple with the id as key
all_nodes=[]          # For storing list of node ids of all nodes
all_elevations=[]     # For storing elevations of all nodes
## MAYBE SAVE ALL NODE IDS IN DATAFRAME WITH ELEVATION AND BASE DEMAND AND THEN FILTER DATA FRAME LATER FOR DEMAND NODES ONLY

# Creates a network model object using EPANET .inp file
network=wntr.network.WaterNetworkModel(abs_path)

# Iterates over the junction list in the Network object
for node in network.junctions():
    all_nodes.append(node[1].name)
    all_elevations.append(node[1].elevation)
    coords[node[1].name]=node[1].coordinates
    # For all nodes that have non-zero demands
    if node[1].base_demand != 0:
        # Record node ID (name), desired demand (base_demand) in CMS, elevations, x and y coordinates
        demand_nodes.append(node[1].name)
        desired_demands.append(node[1].base_demand)
        elevations.append(node[1].elevation)
        

conduit_ids= []       # To store IDs of the original pipes in the EPANET file
conduit_from= []      # To store the origin node for each pipe
conduit_to= []        # To store the destination node for each pipe
conduit_lengths= []   # To store pipe lengths
conduit_diameters= [] # To store pipe diameters

# Loop over each link in the EPANET model
for link in network.links():

    # Extract and store each of the aforementioned properties
    conduit_ids.append(link[1].name)
    conduit_from.append(link[1].start_node_name)
    conduit_to.append(link[1].end_node_name)
    conduit_lengths.append(link[1].length)
    conduit_diameters.append(link[1].diameter)

reservoir_ids=[]      # To store the source reservoirs' IDs
reservoir_heads={}    # To store the total head of each reservoir indexed by ID
reservoir_coords={}   # To store the coordinates as tuple (x,y) indexed by ID

# Loops over each reservoir
for reservoir in network.reservoirs():
    reservoir_ids.append(reservoir[1].name)
    reservoir_heads[reservoir_ids[-1]]=reservoir[1].base_head
    reservoir_coords[reservoir_ids[-1]]=reservoir[1].coordinates
reservoir_elevations={reservoir:reservoir_heads[reservoir]-30 for reservoir in reservoir_heads}

# Get the supply duration in minutes (/60) as an integer
supply_duration=int(network.options.time.duration/60)
supply_hh=str(supply_duration//60)     # The hour value of the supply duration (quotient of total supply in minutes/ 60)
supply_mm=str(supply_duration%60)      # The minute value of the supply duration (remainder)

# Corrects the formatting of the HH:MM by adding a 0 if it is a single digit: if minutes =4 -> 04
if len(supply_mm)<2:
    supply_mm='0'+supply_mm
if len(supply_hh)<2:
    supply_hh='0'+supply_hh

### Spatial Discretization of Longer Pipes
To improve the stability and accuracy of SWMM in complex networks, longer pipes should be discretized into smaller ones  
Refer to [3] for a more in-depth discussion of this procedure  
The following cell breaks down pipes that are longer than a specified maximum $\Delta x_{max}$  
Pipes longer than the maximum length are divided into equal parts, where the number of parts is:   
$$ N_{parts}= \left\lceil \frac{L_{pipe}}{\Delta x_{max}} \right\rceil$$
The length of each part is then set at:
$$L_{part}=\frac{L_{pipe}}{N_{parts}}$$
After creating all pipe segments, intermediate nodes are created to join the pipe segments.  
The elevation of these nodes (as well as the x and y coordinates) are interpolated linearly using the start elevation and the end elevation,  
 where the difference in elevation between each two consecutive nodes set as:
$$\Delta E = \frac{ E_{end}-E_{start}}{N_{parts}}$$
Similarly, the difference in x and y coordinates are found as:  
$$\Delta x = \frac{ x_{end}-x_{start}}{N_{parts}}\\
\Delta y = \frac{ y_{end}-y_{start}}{N_{parts}}$$

In [10]:
# Maximum length of conduit allowed
maximum_xdelta=10

# Dataframe aggregating all node information gathered from the EPANET file
junctions=pd.DataFrame(zip(all_nodes,all_elevations,coords.values()),columns=["ID","Elevation","Coordinates"])
# Set the junction ID as the index of the Dataframe
junctions.set_index("ID",inplace=True)

# Dataframe aggregating all conduit information gathered from the EPANET file
conduits=pd.DataFrame(zip(conduit_ids,conduit_from,conduit_to,conduit_lengths,conduit_diameters),columns=["ID","from node","to node","Length","diameter"])
# Set the conduit ID as the index
conduits.set_index("ID",inplace=True)

# Loop over each conduit in the original file
for conduit in conduits.index:

    length=conduits["Length"][conduit]  #Stores the length of the current conduit for shorthand

    # If the conduit is bigger than the maximum allowable length (delta x), we will break it down into smaller pipes
    if length>maximum_xdelta:
        # Number of smaller pipes is calculated from 
        n_parts=math.ceil(length/maximum_xdelta)
        # Calculate the length of each part 
        part_length=length/n_parts
        # Start node ID (for shorthand)
        start_node=conduits["from node"][conduit]
        # End node ID (for shorthand)
        end_node=conduits["to node"][conduit]
        # If the start node is a reservoir
        if start_node in reservoir_ids:
            # MAke the start elevation the same as the end but add 1 (since reservoirs don't have ground elevation in EPANET)
            start_elevation=junctions.at[end_node,"Elevation"]+1
            reservoir_elevations[start_node]=start_elevation+1
        # Otherwise make the start elevation equal to the elevation of the start node
        else: start_elevation=junctions.at[start_node,"Elevation"]
        
        # If the end node is a reservoir
        if end_node in reservoir_ids:
            # MAke the end elevation the same as the start but subtract 1 (since reservoirs don't have ground elevation in EPANET)
            end_elevation=start_elevation-1
        # Make the end elevation equal to the elevation of the end node
        else: end_elevation=junctions.at[end_node,"Elevation"]
        # Calculate the uniform drop (or rise) in elevation for all the intermediate nodes about to be created when this pipe is broken into several smaller ones
        unit_elev_diff=(end_elevation-start_elevation)/n_parts

        # if the starting node is a reservoir
        if start_node in reservoir_ids:
            # Get coordinates from reservoir data
            start_x=reservoir_coords[start_node][0]
            start_y=reservoir_coords[start_node][1]
        else:
            # Get the coordinates from the junction data
            start_x=junctions.at[start_node,"Coordinates"][0]
            start_y=junctions.at[start_node,"Coordinates"][1]
        
        # If the end node is a reservoir
        if end_node in reservoir_ids:
            # Get the coordinates from the reservoir data
            end_x=reservoir_coords[end_node][0]
            end_y=reservoir_coords[end_node][1]
        else:
            # Get them from the junctions data
            end_x=junctions.at[end_node,"Coordinates"][0]
            end_y=junctions.at[end_node,"Coordinates"][1]
            
        # Calculate the unit difference in x and y coordinates for this pipe and its segments
        unit_x_diff=(end_x-start_x)/n_parts
        unit_y_diff=(end_y-start_y)/n_parts


# THIS LOOP GENERATES THE SMALLER PIPES TO REPLACE THE ORIGINAL LONG PIPE
        # For each part to be created
        for part in np.arange(1,n_parts+1):

            # CREATING THE LINKS
            # Create the ID for the new smaller pipe as OriginPipeID-PartNumber
            new_id=conduit+"-"+str(part)
            # Set the new pipe's diameter equal to the original one
            conduits.at[new_id,"diameter"]=conduits["diameter"][conduit]
            # Set the start node as OriginStartNode-NewNodeNumber-OriginEndNode  as in the first intermediate nodes between node 13 and 14 will be named 13-1-14
            conduits.at[new_id,"from node"]=start_node+"-"+str(part-1)+"-"+end_node
            # if this is the first part, use the original start node 
            if part==1:
                conduits.at[new_id,"from node"]=start_node
            # Set the end node as OriginStartNode-NewNodeNumber+1-OriginEndNode  as in the second intermediate nodes between node 13 and 14 will be named 13-2-14
            conduits.at[new_id,"to node"]=start_node+"-"+str(part)+"-"+end_node
            # If this is the last part, use the original end node as the end node
            if part==n_parts:
                conduits.at[new_id,"to node"]=end_node
            # Set the new pipe's length to the length of each part
            conduits.at[new_id,"Length"]=part_length

            # if this is NOT the last part (as the last pipe segment joins a pre-existing node and does not need a node to be created)
            if part<n_parts:
                # Create a new node at the end of this pipe segment whose elevation is translated from the start elevation using the unit slope and the part number
                junctions.at[conduits.at[new_id,"to node"],"Elevation"]=start_elevation+part*unit_elev_diff
                # Calculate the coordinates for the new node using the unit difference in x and y coordinates
                junctions.at[conduits.at[new_id,"to node"],"Coordinates"]=(start_x+part*unit_x_diff,start_y+part*unit_y_diff)

        # After writing the new smaller pipes, delete the original pipe (since it is now redundant)
        conduits.drop(conduit,inplace=True)

### Writing Junctions
The junction section lines are written. Data required for the junctions include:
Name: Already stored in junctions  
Elevation: Already stored in junctions  
MaxDepth: 0  
Initial Depth (InitDepth): 0 No initialization required  
Surcharge Depth (SurDepth): 100 or any value high enough to prevent the node from overflowing (to simulate a pressurized pipe)  
Area Ponded (Aponded): 0  No ponding

In [11]:
MaxDepth=[0]*len(junctions)
InitDepth=MaxDepth
SurDepth=[100] * len(junctions)  # High value to prevent surcharging
Aponded=InitDepth

# Creates dataframe with each row representing one line from the junctions section
junctions_section=pd.DataFrame(list(zip(junctions.index,junctions["Elevation"],MaxDepth,InitDepth,SurDepth,Aponded)))
# Converts the dataframe into a list of lines in the junctions section
junctions_section=junctions_section.to_string(header=False,index=False,col_space=10).splitlines()
# adds a new line character to the end of each line in the section
junctions_section=[line+'\n' for line in junctions_section]

### Writing Outfalls
VRW in SWMM does not require outfalls, however, SWMM does require at least 1 outfall to perform a simulation  
Thus one artificial outfall is added, the flow to which will be negligibly set by its corresponding outlet
Data required for outfalls:  
Name: formatted as OutfallX where X is the ID of the original demand node  
Elevation: equal to the original demand node's elevation  
Type: FREE no boundary conditions forced  
Stage Data: Blank None provided  
Gated: NO  
Route To: Blank None specified

In [12]:
# print(demand_nodes)
outfall_ids=["Outfall"+ str(node) for node in demand_nodes]
outfall_elevations=elevations
outfall_types=['FREE' for i in demand_nodes]
stage_data=["    " for i in demand_nodes]
outfall_gated=["NO" for i in demand_nodes]
# Creates dataframe with each row representing one line from the outfalls section
outfall_section=pd.DataFrame(zip(outfall_ids,outfall_elevations,outfall_types,stage_data,outfall_gated))
# Converts the dataframe into a list of lines in the outfalls section
outfall_section=outfall_section.to_string(header=False,index=False,col_space=10).splitlines()
# adds a new line character to the end of each line in the section
outfall_section=[line+'\n' for line in outfall_section]

### Writing Storage Nodes
Two types of storage nodes are needed in a VRW SWMM simulation  
First, storage nodes that represent the constant-head sources (reservoirs) present in the original file. These are assigned large constant area values to fix their head at a constant value.  
Second, each demand node is connected to a storage node with a defined volume curve that specifies the area as:  
$$
A(d)=
\begin{cases}
V_{des}/h_{tank} & \quad \text{when}\, 0 \leqslant d \leqslant h_{tank}\\
\approx 0 & \quad \text{otherwise}
\end{cases}
$$
Where $A(d)$ is the Area of the Storage as a function of the water depth $d$, $h_{tank}$ is the specified height of the consumer tank, and $V_{des}$ is the consumer's desired volume in a supply cycle  
In this case we use the height of the consumer tank as 1 m, but feel free to change that if you need.

In [13]:
# Main Storage
tank_height=1

storage_ids=["StorageforNode"+id for id in demand_nodes]
storage_areas=[demand*60* supply_duration/tank_height for demand in desired_demands]
storage_elevations=elevations
storage_curves=[round(volume*10000) for volume in storage_areas]
storage_MaxDepth=[max(100,max(reservoir_heads.values()))]*len(storage_ids)
storage_InitDepth=[0]*len(storage_ids)
storage_shape=["TABULAR"]*len(storage_ids)
blanks=['    ']*len(storage_ids)    # for the other curve parameters which are not required in Tabular definition
storage_SurDepth=[0]*len(storage_ids)
storage_fevap=[0]*len(storage_ids)

storage_units=pd.DataFrame(zip(storage_ids,storage_elevations,storage_MaxDepth,storage_InitDepth,storage_shape,storage_curves,blanks,blanks,storage_SurDepth,storage_fevap))

In [14]:
# Over Flow Storage
ovf_ids=["OverflowforNode"+id for id in demand_nodes]
ovf_area=1
ovf_elevations=elevations
ovf_curves=["Overflow" for volume in storage_areas]
ovf_MaxDepth=[1]*len(storage_ids)
ovf_InitDepth=[0]*len(storage_ids)
ovf_shape=["TABULAR"]*len(storage_ids)
blanks=['    ']*len(storage_ids)    # for the other curve parameters which are not required in Tabular definition
ovf_SurDepth=[0]*len(storage_ids)
ovf_fevap=[0]*len(storage_ids)

ovf_units=pd.DataFrame(zip(ovf_ids,ovf_elevations,ovf_MaxDepth,ovf_InitDepth,ovf_shape,ovf_curves,blanks,blanks,ovf_SurDepth,ovf_fevap))

In [15]:
reservoir_ids_new=["Reservoir-"+str(id) for id in reservoir_ids]
for i in range(0,len(reservoir_ids)):
    conduits.replace(to_replace=reservoir_ids[i],value=reservoir_ids_new[i])
reservoir_elevations=reservoir_elevations.values()
MaxDepth=[max(100,max(reservoir_heads.values())+10)]*len(reservoir_ids)
InitDepth=[head-elevation for head, elevation in zip(reservoir_heads.values(),reservoir_elevations)]
reservoir_shape=["FUNCTIONAL"]*len(reservoir_ids)
reservoir_coeff=[0]*len(reservoir_ids)
reservoir_expon=[0]*len(reservoir_ids)
reservoir_const=[1000000]*len(reservoir_ids)
reservoir_SurDepth=reservoir_expon
reservoir_psi=reservoir_expon

storage_section=pd.DataFrame(zip(reservoir_ids,reservoir_elevations,MaxDepth,InitDepth,reservoir_shape,reservoir_coeff,reservoir_expon,reservoir_const,reservoir_SurDepth,reservoir_psi))
storage_section= pd.concat([storage_section,storage_units,ovf_units])
storage_section=storage_section.to_string(header=False,index=False,col_space=10).splitlines()
storage_section=[line+'\n' for line in storage_section]

### Writing Conduits
Name From To L Roughness(manning) InOff  OutOff InitFlow  MaxFlow

In [16]:
roughness=[0.011]*len(conduits)
conduit_zeros=[0]*len(conduits)

conduits_section=pd.DataFrame(zip(conduits.index,conduits["from node"],conduits["to node"],conduits["Length"],roughness,conduit_zeros,conduit_zeros,conduit_zeros,conduit_zeros))
conduits_section=conduits_section.to_string(header=False,index=False,col_space=10).splitlines()
conduits_section=[line+'\n' for line in conduits_section]

### Writing Outlets
To have a rating curve with no upper limit, we use the FUNCTIONAL mode of definition in relation to the depth (Pressure) at the original demand node  
Functional definition of rating curves is specified as:
$$ Q_{outlet}=c d^{\alpha}$$
where d is teh depth, c is a coefficient and alpha is an exponent  
We aim to approximate the head-flow relationship (above) using this definition, by using the same exponent 0.5 and calculating the coefficient such that:  
$$ Q=Q_{des} \text{ when } d=H^{des}-H^{min}$$
Thus the function's coefficient can be found as:  
$$ c=\frac{Q_{des}}{\sqrt{H^{des}-H^{min}}}$$
Since the depth at the node is equal to $H_j$, this approximation is exact when $H_{min}=0$ 
$$ Q_{outlet}=Q_{des}\sqrt{\frac{H_j}{{H^{des}-H^{min}}}}$$
Lastly, an outlet is needed to restrict the flow to the artificial outfall to a negligible amount by setting the coefficient to a sufficiently small value and the exponent to 0

In [17]:
# Main Outlet
outlet_ids = ["Outlet"+id for id in demand_nodes]
outlet_from = demand_nodes[:]
outlet_to = storage_ids [:]
outlet_offset=[0]*len(outlet_ids)
outlet_type=["FUNCTIONAL/DEPTH"]*len(outlet_ids)
outlet_coeff=[demand*1000/np.sqrt(pressure_diff) for demand in desired_demands]  # To generate unique Table IDs for each demand rate (not demand node) i.e., juncitons with the same demand are assigned the same outlet curve
outlet_expon=["0.5"]*len(outlet_ids)
outlet_gated=["YES"]*len(outlet_ids)

# Overflow Outlet
outovf_ids = ["OverflowOutlet"+id for id in demand_nodes]
outovf_from = storage_ids[:]
outovf_to = ovf_ids [:]
outovf_offset=[tank_height]*len(outlet_ids)
outovf_type=["TABULAR/DEPTH"]*len(outlet_ids)
outovf_coeff=["Drain"]*len(outovf_ids) 
outovf_expon=["     "]*len(outlet_ids)
outovf_gated=["YES"]*len(outlet_ids)

# Demand Outlet
outdemand_ids = ["DemandOutlet"+id for id in demand_nodes]
outdemand_from = storage_ids [:]
outdemand_to = outfall_ids [:]
outdemand_offset=[0]*len(outlet_ids)
outdemand_type=["TABULAR/DEPTH"]*len(outlet_ids)
outdemand_coeff=["Demand"+ id for id in demand_nodes]
outdemand_expon=["     "]*len(outlet_ids)
outdemand_gated=["YES"]*len(outlet_ids)

outlets=pd.DataFrame(list(zip(outlet_ids,outlet_from,outlet_to,outlet_offset,outlet_type,outlet_coeff,outlet_expon,outlet_gated)))
outletovfs=pd.DataFrame(list(zip(outovf_ids,outovf_from,outovf_to,outovf_offset,outovf_type,outovf_coeff,outovf_expon,outovf_gated)))
outletdemand=pd.DataFrame(list(zip(outdemand_ids,outdemand_from,outdemand_to,outdemand_offset,outdemand_type,outdemand_coeff,outdemand_expon,outdemand_gated)))
outlet_section=pd.concat([outlets,outletovfs,outletdemand])
outlet_section=outlet_section.to_string(header=False,index=False,col_space=10).splitlines()
outlet_section=[line+'\n' for line in outlet_section]

### Writing Pumps

In [18]:
pump_ids = ["Pump" + id for id in demand_nodes]
pump_from = ovf_ids
pump_to = demand_nodes
pump_curves = ["OVERFLOW_PUMP"]*len(pump_ids)
pump_status = ["ON"]*len(pump_ids)
pump_startup = [0]*len(pump_ids)
pump_shutoff = [0]*len(pump_ids)

pumps=pd.DataFrame(list(zip(pump_ids,pump_from,pump_to,pump_curves,pump_status,pump_startup,pump_shutoff)))
pumps_section=pumps.to_string(header=False,index=False,col_space=10).splitlines()
pumps_section=[line+'\n'for line in pumps_section]


### Writing X-Sections
Link    Shape     Geom1 (DIAMETER)    Geom2 (HW Coefficient)     Geom3    Geom 4       Barrels     Culvert (EMPTY)

In [19]:
shape=["FORCE_MAIN"]*len(conduits.index)
hwcoeffs=[130]*len(shape)
geom3=[0]*len(shape)
geom4=geom3
nbarrels=[1]*len(shape)

xsections_section=pd.DataFrame(zip(conduits.index,shape,conduits["diameter"],hwcoeffs,geom3,geom4,nbarrels))
xsections_section=xsections_section.to_string(header=False,index=False, col_space=10).splitlines()
xsections_section=[line+'\n' for line in xsections_section]

### Writing Curves
Storage Curves  
Outlet Curves  
Pump Curves

In [20]:
table_ids=list(set(storage_curves))   # removes duplicates from list
curves_name=[]
curves_type=[]
curves_x=[]
curves_y=[]
for table in table_ids:
    volume=int(table)/10000                # in LPS
    for depth in [0,1,1.0001,100]:
        curves_name.append(table)
        curves_x.append(depth)
        if depth<=1:
            curves_y.append(volume)
        else: curves_y.append(1)
        if depth==0:
            curves_type.append("Storage")
        else: 
            curves_type.append(" ")
    curves_name.append(";")
    curves_type.append(" ")
    curves_x.append(" ")
    curves_y.append(" ")
curves_name+=["Overflow","Overflow",';']
curves_type+=["Storage","   ",';']
curves_x+=[0,1,';']
curves_y+=[1,1,';']


curves_name+=["OVERFLOW_PUMP","OVERFLOW_PUMP",';']
curves_type+=["Pump4","   ",' ']
curves_x+=[0,1,' ']
curves_y+=[0,1000,' ']

curves_name+=["Drain","Drain",';']
curves_type+=["Rating","   ",' ']
curves_x+=[0,1,' ']
curves_y+=[0,1000,' ']

constant_demands=[demand*float(supply_hh)/24 for demand in desired_demands]
for i,j in zip(demand_nodes,constant_demands):
    curves_name+=["Demand"+str(i),"Demand"+str(i),';']
    curves_type+=["Rating"," ","  "]
    curves_x+=[0,0.01," "]
    curves_y+=[0,j*1000," "]

curves=pd.DataFrame(list(zip(curves_name,curves_type,curves_x,curves_y)))
curves_section=curves.to_string(header=False,index=False,col_space=10).splitlines()
curves_section=[line+'\n' for line in curves_section]


### Writing Coordinates

In [21]:
coords_demand= { node: coords[node] for node in demand_nodes}
coords_ids=list(junctions.index)+reservoir_ids+storage_ids+ovf_ids+outfall_ids

coords_x1=[coord[0] for coord in junctions["Coordinates"]]
coords_x2=[coord[0] for coord in reservoir_coords.values()]
coords_x3=[coord[0] +20 for coord in coords_demand.values()]
coords_x4=[coord[0] +10 for coord in coords_demand.values()]
coords_x5=[coord[0] +30 for coord in coords_demand.values()]
coords_x=coords_x1+coords_x2+coords_x3+coords_x4+coords_x5

coords_y1=[coord[1] for coord in junctions["Coordinates"]]
coords_y2=[coord[1] for coord in reservoir_coords.values()]
coords_y3=[coord[1] for coord in coords_demand.values()]
coords_y4=[coord[1]+10 for coord in coords_demand.values()]
coords_y5=[coord[1] for coord in coords_demand.values()]
coords_y=coords_y1+coords_y2+coords_y3+coords_y4+coords_y5

coordinate_section=pd.DataFrame(zip(coords_ids,coords_x,coords_y))
coordinate_section=coordinate_section.to_string(header=False,index=False,col_space=10).splitlines()
coordinate_section=[line+'\n' for line in coordinate_section]

#Setting View Dimensions
x_left=min(coords_x)-max(coords_x)/4
x_right=max(coords_x)+max(coords_x)/4
y_down=min(coords_y)-max(coords_y)/4
y_up=max(coords_y)+max(coords_y)/4
dimensions_line=str(x_left)+" "+str(y_down)+" "+str(x_right)+" "+str(y_up)+"\n"

### Creating the SWMM .inp File

In [25]:
# opens .inp file to read
file=open(r"C:\IWS_Modelling\Github\IWS-Modelling-Methods-Repo\Network-Files\EMPTY_SWMM_TEMPLATE.inp",'r')
lines=[]              # list to store all lines in the .inp file
linecount=0           # Counter for the number of lines

# Loops over each line in the input file 
for line in file:
    if re.search("^END_TIME",line):
        end_time=linecount
    if re.search("^DIMENSIONS",line):
        dimensions=linecount
    # Record the position of the phrase [JUNCTIONS] and add 3 to skip the header lines
    if re.search('\[JUNCTIONS\]',line):
        junctions_marker=linecount+3
    # Record the position of the phrase [OUTFALLS] and add 3 to skip the header lines
    if re.search('\[OUTFALLS\]',line):
        outfalls_marker=linecount+3
    # Record the position of the phrase [STORAGE] and add 3 to skip the header lines
    if re.search('\[STORAGE\]',line):
        storage_marker=linecount+3
    # Record the position of the phrase [CONDUITS] and add 3 to skip the header lines
    if re.search('\[CONDUITS\]',line):
        conduits_marker=linecount+3
     # Record the position of the phrase [OUTLETS] and add 3 to skip the header lines
    if re.search('\[PUMPS\]',line):
        pumps_marker=linecount+3
    if re.search('\[OUTLETS\]',line):
        outlets_marker=linecount+3
     # Record the position of the phrase [XSECTIONS] and add 3 to skip the header lines
    if re.search('\[XSECTIONS\]',line):
        xsections_marker=linecount+3
    # Record the position of the phrase [CURVES] and add 3 to skip the header lines
    if re.search('\[CURVES\]',line):
        curves_marker=linecount+3
    # Record the position of the phrase [COORDINATES] and add 3 to skip the header lines
    if re.search('\[COORDINATES\]',line):
        coords_marker=linecount+3
    # Store all lines in a list
    lines.append(line)
    linecount+=1
file.close()

In [26]:
file=open(directory+name_only+'_3O2S1P.inp','w')
lines[end_time]="END_TIME             "+str(23)+":"+str(59)+":00\n"
lines[dimensions]="DIMENSIONS "+dimensions_line
lines[coords_marker:coords_marker]=coordinate_section
lines[curves_marker:curves_marker]=curves_section
lines[xsections_marker:xsections_marker]=xsections_section
lines[outlets_marker:outlets_marker]=outlet_section
lines[pumps_marker:pumps_marker]=pumps_section
lines[conduits_marker:conduits_marker]=conduits_section
lines[storage_marker:storage_marker]=storage_section
lines[outfalls_marker:outfalls_marker]=outfall_section
lines[junctions_marker:junctions_marker]=junctions_section



# All lines added by this script are missing a new line character at the end, the conditional statements below add the new line character for these lines only and writes all lines to the file
for line in lines:
    file.write(line)    
file.close()